In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
from time import time
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import cross_validate, ParameterGrid, GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelBinarizer
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import make_scorer, accuracy_score, classification_report

In [ ]:
def run_report():
    time1 = time()
    print("Before beginninng, make sure that your CSV is in your juypter notebook folder.")
    print("If it is not, kill the kernel now.")
    csv = (input("Please enter the CSV name: ") + ".csv")
    #if this gets hooked up with GCP, then set the query name equal to csv
    new_csv = input("Please name output CSV: ")
    
    global df_original
    global df
    df_original = pd.read_csv(csv)
    df = pd.read_csv(csv)
    df["date_today"] = datetime.now()
    df = df.dropna(subset = ["original_estimate_complete_datetime"])
    df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
    df = df.assign(time_since_estimate =  df['date_today'] - df['estimate_date_parse'])
    df["days_since_estimate"] = df["time_since_estimate"].apply(lambda x: int(str(x).split()[0]))
   
    del df["time_since_estimate"]
    df1 = df[df["suppCnt"] == 0]
    df2 = df[df["suppCnt"] != 0]
    df1["suppInd"] = 0
    df2["suppInd"] = 1
    df = pd.concat([df1,df2], axis = 0)
    df["paid_loss_amount"] = df["paid_loss_amount"].apply(lambda x: abs(x))
    df["primary_point_of_impact"].fillna("Unknown",inplace = True)
    df["total_loss_points"].fillna(999,inplace = True)
    df["insured_fault_pct"].fillna(999,inplace = True)
    global df_w_days
    df_w_days = df
    df = df[["claim_number","company","coverage_grouping","detailed_loss_cause",'driveable_ind',"insured_fault_pct","insured_fault_rating","line_of_business","loss_cause","loss_party_type","loss_state","original_estimate_complete_datetime","payAfterCloseInd","policy_state","policy_type","primary_point_of_impact","saved_vehicle_as_type","secondary_point_of_impact","total_loss_points","vehicle_make","vehicle_year","suppInd","maxNet","minNet","maxFile","minFile","days_since_estimate"]]
    global df_full_data
    df_full_data = df
    df = df[df["days_since_estimate"] >= 90]
    df = df.set_index("claim_number")
    
    target_name = "suppInd"
    Y = df[target_name]
    X = df.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate","maxNet","minNet","company","line_of_business","policy_state",])
    
    numerical_columns_selector = selector(dtype_exclude=object)
    categorical_columns_selector = selector(dtype_include=object)
    numerical_columns = numerical_columns_selector(X)
    categorical_columns = categorical_columns_selector(X)
    categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
    numerical_preprocessor = StandardScaler()
    preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])
    
    model = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth = 7))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42, train_size = 0.75)
    global clf
    clf = model.fit(X_train, Y_train)
    accuracy = clf.score(X_test, Y_test)
    print(len(X_train))
    
    df_under_90 = df_full_data[df_full_data["days_since_estimate"] < 90]
    print(len(df_under_90))
    df_under_90["prediction"] = clf.predict(df_under_90)
    df_under_90["prediction1"] = clf.predict(df_under_90)
    df_under_90["prediction2"] = clf.predict(df_under_90)
    df_under_90["prediction3"] = clf.predict(df_under_90)
    
    predict_percent_90 = len(df_under_90[df_under_90["prediction"] == 1]) / len(df_under_90)
    total_percent_supp = len(df_under_90[df_under_90["suppInd"] == 1][df_under_90["prediction"] == 1]) / len(df_under_90)
    sup_over_90 = len(df[df["suppInd"] == 1])/len(df)
    global flagged_claims
    flagged_claims = df_under_90[df_under_90["prediction"] == 1][df_under_90["suppInd"] == 0][["claim_number"]]
    print(len(flagged_claims))
    report = "Accuracy of Data: " + str(accuracy) + "\n" + "Percent of claims that have supplements over 90 days: " + str(sup_over_90) + "\n" + "Percent of claims that have predicted supplements under 90 days: " +  str(predict_percent_90)  
    print(report)
    flagged_claims_full_data = pd.merge(flagged_claims, df_original, how = "left", on = "claim_number").drop_duplicates()
    
    flagged_claims_full_data.to_csv((new_csv + '.csv'), index=False, encoding='utf-8')  
    
    new_supp_count = len(flagged_claims_full_data)
    
    print( "Count of claims that have predicted new supplements: " + str(new_supp_count))
    time2 = time()
    print("Runtime: " + str(round(time2 - time1, 3)) + " seconds")

    return(df_under_90)

In [27]:
df1 = run_report()

Before beginninng, make sure that your CSV is in your juypter notebook folder.
If it is not, kill the kernel now.
Please enter the CSV name: Supp Prediction
Please name output CSV: Flagged Claims


C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:10: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:11: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:21: SettingWithCopyWarning: 
A value is trying to be set 

152042
83532


C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under_90["prediction"] = clf.predict(df_under_90)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under_90["prediction1"] = clf.predict(df_under_90)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17724\3621338803.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

11402
Accuracy of Data: 0.8386180225330991
Percent of claims that have supplements over 90 days: 0.5372010082723716
Percent of claims that have predicted supplements under 90 days: 0.37631087487429965
Count of claims that have predicted new supplements: 12794
Runtime: 117.852 seconds


In [28]:
df1

,claim_number,company,coverage_grouping,detailed_loss_cause,driveable_ind,insured_fault_pct,insured_fault_rating,line_of_business,loss_cause,loss_party_type,...,suppInd,maxNet,minNet,maxFile,minFile,days_since_estimate,prediction,prediction1,prediction2,prediction3
1,01-004-717683,Connect,Collision,Insured Rear-ended Claimant,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,0,4256.30,4256.30,4756.30,4756.30,68,0,0,0,0
6,01-004-683118,Amfam,Collision,Claimant Rearended Insured,No,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,0,12175.37,12175.37,12675.37,12675.37,88,0,0,0,0
17,01-004-715340,Amfam,Collision,Collision While Turning,No,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,0,20196.45,20196.45,20446.45,20446.45,84,0,0,0,0
19,01-004-992800,Midvale,Collision,Ditch,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Roadbed,Insured's loss,...,0,33124.19,33124.19,33624.19,33624.19,24,0,0,0,0
30,01-004-609862,Amfam,Property Damage Liability,Head On Collision,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Third-party liability,...,0,5416.51,5416.51,5416.51,5416.51,83,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286580,01-004-820214,Amfam,Collision,Collision While Turning,No,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,1,4301.08,2138.86,4801.08,2638.86,53,1,1,1,1
286588,01-004-788524,Amfam,Property Damage Liability,Backing/Reversing,Yes,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Third-party liability,...,1,2241.86,1672.09,2241.86,1672.09,39,0,0,0,0
286592,01-004-818419,Amfam,Comprehensive,Engine Fire,No,0.0,No fault,Personal Auto Line,Fire,Insured's loss,...,1,-250.00,-250.00,0.00,0.00,69,0,0,0,0
286598,01-004-746012,Amfam,Comprehensive,NaN,Yes,0.0,No fault,Personal Auto Line,Hail,Insured's loss,...,1,5890.51,2142.90,6390.51,2642.90,81,1,1,1,1


In [5]:
df = df_full_data[df_full_data["days_since_estimate"] >= 90]

In [6]:
df_under_90 = df_full_data[df_full_data["days_since_estimate"] < 90]

In [10]:
list(clf.predict(df_under_90)).count(1)

30433

In [9]:
clf.predict(df_under_90)

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)